<a href="https://colab.research.google.com/github/Konstantin5054232/ausbildungsprojekte/blob/main/01_musik_der_gro%C3%9Fst%C3%A4dte/musik_der_gro%C3%9Fst%C3%A4dte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Musik der Großstädte

Der Musikdienst lieferte Daten über die von Nutzern in Moskau und St. Petersburg abgehörten Tracks für einen bestimmten Zeitraum.
Es ist notwendig, Fragen zu beantworten und Hypthesen zu überprüfen:
- Hören Menschen in verschiedenen Städten wirklich unterschiedlich Musik?
- Montagmorgen und Freitagabend - andere Musik oder die gleiche?
- Bevorzugen die Bewohner der beiden Städte unterschiedliche Musikrichtungen?

In [6]:
# Wir importieren die notwendigen Bibliotheken.
import pandas as pd

In [7]:
# Wir laden Tabellen mit Daten.
df = pd.read_csv('/music_project.csv')

In [8]:
# Wir werden die erhaltenen Daten studieren
df.info()
display(df.shape)
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


(65079, 7)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday


**Schlussfolgerungen**

Jede Zeile der Tabelle enthält Informationen zu Songs eines bestimmten Genres in einer bestimmten Performance, die Benutzer zu bestimmten Zeiten und Wochentagen in einer der Städte angehört haben. Zwei Probleme, die angegangen werden müssen: fehlende und schlechte Spaltennamen. Die Spalten time, day und City sind besonders wertvoll, um Arbeitshypothesen zu testen. Mit den Daten aus der Spalte genre können Sie die beliebtesten Genres kennenlernen.

# Datenvorverarbeitung

In [9]:
# Wir erhalten eine Liste der Spaltennamen.
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

In [10]:
# Wir werden die Spalten umbenennen, um die weitere Arbeit zu erleichtern.
df.set_axis(['user_id', 'track_name', 'artist_name', 'genre_name', 'city', 'time', 'weekday'], 
            axis = 'columns',inplace = True)

# Mal sehen was passiert
df.columns

Index(['user_id', 'track_name', 'artist_name', 'genre_name', 'city', 'time',
       'weekday'],
      dtype='object')

In [11]:
# Wir werden prüfen, ob es Nullwerte gibt.
df.isnull().sum()

user_id           0
track_name     1231
artist_name    7203
genre_name     1198
city              0
time              0
weekday           0
dtype: int64

In [12]:
# Wir ersetzen die fehlenden Werte in den Spalten mit dem Titel und dem Interpreten durch die Zeile 'unknown'.
df['artist_name']=df['artist_name'].fillna('unknown')
df['track_name']=df['track_name'].fillna('unknown')

In [13]:
# Wir entfernen leere Werte in der Spalte mit den Genres.
df.dropna(subset = ['genre_name'], inplace = True)

In [14]:
# Mal sehen was passiert
df.isnull().sum()

user_id        0
track_name     0
artist_name    0
genre_name     0
city           0
time           0
weekday        0
dtype: int64

In [15]:
# Wir werden prüfen, ob es Duplikate gibt.
df.duplicated().sum()

3755

In [16]:
# Wir werden Duplikate entfernen.
df = df.drop_duplicates().reset_index(drop=True)

# Mal sehen was passiert
df.duplicated().sum()

0

In [17]:
# Wir speichern die Liste der eindeutigen Werte einer Spalte mit Genres in der Variablen genres_list.
genres_list = df['genre_name'].unique() 
genres_list

array(['rock', 'pop', 'folk', 'dance', 'rusrap', 'ruspop', 'world',
       'electronic', 'alternative', 'children', 'rnb', 'hip', 'jazz',
       'postrock', 'latin', 'classical', 'metal', 'reggae', 'tatar',
       'blues', 'instrumental', 'rusrock', 'dnb', 'türk', 'post',
       'country', 'psychedelic', 'conjazz', 'indie', 'posthardcore',
       'local', 'avantgarde', 'punk', 'videogame', 'techno', 'house',
       'christmas', 'melodic', 'caucasian', 'reggaeton', 'soundtrack',
       'singer', 'ska', 'shanson', 'ambient', 'film', 'western', 'rap',
       'beats', "hard'n'heavy", 'progmetal', 'minimal', 'contemporary',
       'new', 'soul', 'holiday', 'german', 'tropical', 'fairytail',
       'spiritual', 'urban', 'gospel', 'nujazz', 'folkmetal', 'trance',
       'miscellaneous', 'anime', 'hardcore', 'progressive', 'chanson',
       'numetal', 'vocal', 'estrada', 'russian', 'classicmetal',
       'dubstep', 'club', 'deep', 'southern', 'black', 'folkrock',
       'fitness', 'french', 'd

In [18]:
# Wir deklarieren die Funktion find_genre(), um nach impliziten Duplikaten in einer Spalte mit Genres zu suchen. 
# Zum Beispiel, wenn der Name des gleichen Genres in verschiedenen Wörtern geschrieben ist.
def find_genre(genre_search):
    count_genre = 0
    for genre in genres_list:
        if genre == genre_search:
            count_genre += 1
    return count_genre

In [19]:
# Wir rufen die Funktion find_genre() auf, um nach verschiedenen Varianten 
# des Hip-Hop-Genres in der Tabelle zu suchen.
find_genre('hip')

1

In [20]:
find_genre('hop')

0

In [21]:
find_genre('hip-hop')

0

In [22]:
find_genre('hiphop')

1

In [23]:
# Wir deklarieren die Funktion find_hip_hop(), die den falschen Namen dieses Genres 
# in der Spalte 'genre_name' durch 'hip hop' ersetzt und überprüft, ob die Ersetzung erfolgreich war.
def find_hip_hop(df, wrong):
    df['genre_name'] = df['genre_name'].replace(wrong,'hiphop')
    final = df[df['genre_name'] == wrong]['genre_name'].count()
    return final

In [24]:
# Wir rufen die Funktion find_hip_hop() für unsere Tabelle auf
find_hip_hop(df, 'hip')

0

In [25]:
# Mal sehen was passiert
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60126 entries, 0 to 60125
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      60126 non-null  object
 1   track_name   60126 non-null  object
 2   artist_name  60126 non-null  object
 3   genre_name   60126 non-null  object
 4   city         60126 non-null  object
 5   time         60126 non-null  object
 6   weekday      60126 non-null  object
dtypes: object(7)
memory usage: 3.2+ MB


**Schlussfolgerungen**

In der Vorverarbeitungsphase wurden Lücken, Probleme mit Spaltennamen und Duplikate in den Daten festgestellt. Die Lücken sind ausgefüllt, die Duplikate wurden entfernt, die Spaltennamen sind jetzt korrekt und bequem für die weitere Arbeit.
Wir haben nicht begonnen, alle Zeilen mit fehlenden Werten zu löschen, da sie die Namen von Genres enthalten, die uns für die Analyse wichtig sind.

# Hören Menschen in verschiedenen Städten wirklich unterschiedlich Musik?

Es wurde die Hypothese aufgestellt, dass Nutzer in Moskau und St. Petersburg unterschiedlich Musik hören. Wir überprüfen diese Annahme anhand der Daten an drei Tagen der Woche - Montag, Mittwoch und Freitag.

Wir legen für jede Stadt die Anzahl der heute gehörten Songs eines bekannten Genres fest und vergleichen die Ergebnisse.

In [26]:
# Wir gruppieren die Daten nach Stadt und 
# zählen durch Aufruf der Methode count() die Kompositionen, 
# für die das Genre bekannt ist.
df.groupby('city')['genre_name'].count()

city
Moscow              41892
Saint-Petersburg    18234
Name: genre_name, dtype: int64

In [27]:
# Wir gruppieren die Daten nach Wochentagen und zählen die am Montag, 
# Mittwoch und Freitag gehörten Songs, für die das Genre bekannt ist.
df.groupby('weekday')['genre_name'].count()

weekday
Friday       21482
Monday       20866
Wednesday    17778
Name: genre_name, dtype: int64

In [28]:
# Wir erstellen eine number_tracks()-Funktion, die eine Tabelle, einen Wochentag und 
# einen Städtenamen als Parameter verwendet und die Anzahl der gehörten Tracks zurückgibt, 
# für die das Genre bekannt ist. Wir überprüfen die Anzahl der gehörten Songs für jede Stadt 
# und Montag, dann Mittwoch und Freitag.
def number_tracks(df, day, city):
    track_list = df[(df['weekday'] == day) & (df['city'] == city)]
    track_list_count = track_list['genre_name'].count()
    return track_list_count

In [29]:
number_tracks(df, 'Monday', 'Moscow')

15347

In [30]:
number_tracks(df, 'Monday', 'Saint-Petersburg')

5519

In [31]:
number_tracks(df, 'Wednesday', 'Moscow')

10865

In [32]:
number_tracks(df, 'Wednesday', 'Saint-Petersburg')

6913

In [33]:
number_tracks(df, 'Friday', 'Moscow')

15680

In [34]:
number_tracks(df, 'Friday', 'Saint-Petersburg')

5802

In [37]:
# Wir werden die erhaltenen Informationen in einer Tabelle zusammenfassen, 
# wobei ['Stadt', 'Montag', 'Mittwoch', 'Freitag'] die Namen der Spalten sind.
data = [['Moscow', 15347, 10865, 15680],
       ['Saint-Petersburg', 5519, 6913, 5802]]
columns = ['city', 'monday', 'wednesday', 'friday']
table = pd.DataFrame(data = data, columns = columns)
table

,city,monday,wednesday,friday
0,Moscow,15347,10865,15680
1,Saint-Petersburg,5519,6913,5802


**Schlussfolgerungen**

Die Ergebnisse zeigen, dass die Menschen in St. Petersburg und Moskau am Mittwoch Musik „spiegelverkehrt“ hören: In Moskau fallen die Spitzen am Montag und am Freitag, und am Mittwoch nimmt die Hörzeit ab. Während in St. Petersburg der Mittwoch der Tag des größten Musikinteresses ist, ist es am Montag und Freitag weniger und fast gleich weniger.

# Montagmorgen und Freitagabend - andere Musik oder die gleiche?

Wir suchen eine Antwort auf die Frage, welche Genres am Montagmorgen und am Freitagabend in verschiedenen Städten vorherrschen. Es wird davon ausgegangen, dass Benutzer am Montagmorgen belebendere Musik (z. B. Pop-Genre) und am Freitagabend mehr Tanzmusik (z. B. elektronische Musik) hören.

In [38]:
# Wir erhalten Datentabellen für Moskau moscow_general und St. Petersburg spb_general.
moscow_general = df[df['city']=='Moscow']
spb_general = df[df['city']=='Saint-Petersburg']

In [39]:
# Wir erstellen eine Funktion genre_weekday(), die eine Liste von Genres für den angeforderten Wochentag 
# und die Tageszeit von dieser und jener Stunde bis zu dieser und jener zurückgibt.
def genre_weekday(df, day, time1, time2):
    genre_list = df[(df['weekday']==day) & (df['time']>time1) & (df['time']<time2)]
    genre_count = genre_list.groupby('genre_name')['genre_name'].count()
    genre_list_sorted = genre_count.sort_values(ascending = False).head(10)
    return genre_list_sorted

In [41]:
# Wir vergleichen die Ergebnisse, die gemäß der Tabelle für Moskau und St. Petersburg 
# am Montagmorgen (von 7 bis 11) und am Freitagabend (von 17 bis 23) erhalten wurden.
display(genre_weekday(moscow_general, 'Monday', '07:00:00', '11:00:00'))
display(genre_weekday(spb_general, 'Monday', '07:00:00', '11:00:00'))
display(genre_weekday(moscow_general, 'Friday', '17:00:00', '23:00:00'))
display(genre_weekday(spb_general, 'Friday', '17:00:00', '23:00:00'))

genre_name
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
classical      157
Name: genre_name, dtype: int64

genre_name
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre_name, dtype: int64

genre_name
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre_name, dtype: int64

genre_name
pop            256
rock           216
electronic     216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre_name, dtype: int64

**Schlussfolgerungen**

Populäre Genres stellten sich am Montagmorgen in St. Petersburg und Moskau als ähnlich heraus: Überall ist erwartungsgemäß Pop angesagt. Trotzdem ist das Ende der Top 10 für die beiden Städte unterschiedlich: In St. Petersburg gehören Jazz und russischer Rap zu den Top 10, in Moskau das Weltgenre.

Am Ende der Woche ändert sich die Situation nicht. Popmusik steht immer noch an erster Stelle. Auch hier macht sich der Unterschied erst am Ende der Top 10 bemerkbar, wo das Weltgenre am Freitagabend auch in St. Petersburg präsent ist.

Das Pop-Genre ist unangefochten führend, und die Top 5 unterscheiden sich im Allgemeinen nicht in beiden Hauptstädten. Gleichzeitig wird klar, dass das Ende der Liste eher „live“ ist: Für jede Stadt werden charakteristischere Genres unterschieden, die je nach Wochentag und Uhrzeit ihre Position wirklich ändern.

# Bevorzugen die Bewohner der beiden Städte unterschiedliche Musikrichtungen?

In [42]:
# Wir werden die Tabelle moscow_general nach Genre gruppieren, 
# die Anzahl der Songs jedes Genres mit der Methode count() zählen, 
# in absteigender Reihenfolge sortieren und das Ergebnis in der Tabelle moscow_genres speichern.
moscow_genres = moscow_general.groupby('genre_name')['genre_name'].count().sort_values(ascending = False)
moscow_genres.head(10)

genre_name
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre_name, dtype: int64

In [43]:
# Wir werden die Tabelle spb_general nach Genre gruppieren, 
# die Anzahl der Songs jedes Genres mit der Methode count() zählen, 
# in absteigender Reihenfolge sortieren und das Ergebnis in der Tabelle spb_genres speichern.
spb_genres = spb_general.groupby('genre_name')['genre_name'].count().sort_values(ascending = False)
spb_genres.head(10)

genre_name
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre_name, dtype: int64

**Schlussfolgerungen**

Wir sehen, dass die ersten 5 Positionen der Genres in beiden Städten gleich sind. Die Genres auf den Plätzen sechs bis zehn sind ebenfalls gleich, nehmen aber unterschiedliche Positionen ein.

# Forschungsergebnisse

Antworten auf Fragen:

Musik in zwei Städten - Moskau und St. Petersburg - hören die Bewohner in einem anderen Modus;

die Listen der zehn beliebtesten Genres am Montagmorgen und Freitagabend weisen charakteristische Unterschiede auf;

die Bevölkerung der beiden Städte bevorzugt unterschiedliche Musikrichtungen.

**Gesamtergebnis**

Geschmacklich sind sich Moskau und St. Petersburg einig: Überall herrscht populäre Musik. Gleichzeitig gibt es keine Abhängigkeit von Präferenzen vom Wochentag in jeder einzelnen Stadt - die Menschen hören ständig, was ihnen gefällt. Aber zwischen den Städten, was die Wochentage betrifft, gibt es ein Spiegelbild zum Mittwoch: Moskau hört am Montag und Freitag mehr zu, während St. Petersburg im Gegenteil am Mittwoch mehr, aber am Montag und weniger zuhört Freitag.